### Metpiya Lertakkakorn 6410422005

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import Libraries ที่สำคัญ

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from scipy.stats import zscore
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier

สร้าง dataframe เพื่อรับข้อมูล train & test set เพื่อใช้ในการสร้างตัวแบบ

In [ ]:
df_train = pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')
df_train.head()

In [ ]:
df_test = pd.read_csv('/kaggle/input/dads6003-in-class-competition/test.csv')
df_test.head()

In [ ]:
df_train.shape, df_test.shape

## 1. Cleansing data

แบ่งข้อมูล train set ออกเป็น feature & label เพื่อจัดการข้อมูลก่อนนำไปสร้างตัวแบบ

In [ ]:
x_train = df_train.loc[:, 'x1':'x20']
x_train.head()

In [ ]:
y_train = df_train.y
y_train.head()

In [ ]:
len(y_train)

In [ ]:
x_train.info()

In [ ]:
x_train.isnull().sum()

พบว่ายังมีหลาย column ที่ยังมีค่าช่องว่าง คือมีไม่ครบ 7,500 rows จากข้อมูล features ทั้งหมด<br>
จึงสร้างกราฟ histogram เพื่อดูการแจกแจงของข้อมูลแต่ละ features ก่อนจะเลือกวิธีการจัดการกับข้อมูลที่เป็นช่องว่าง

In [ ]:
fig, axes = plt.subplots(5, 4,figsize=(20, 20))
sns.histplot(ax=axes[0,0], x=x_train.x1)
sns.histplot(ax=axes[0,1], x=x_train.x2)
sns.histplot(ax=axes[0,2], x=x_train.x3)
sns.histplot(ax=axes[0,3], x=x_train.x4)
sns.histplot(ax=axes[1,0], x=x_train.x5)
sns.histplot(ax=axes[1,1], x=x_train.x6)
sns.histplot(ax=axes[1,2], x=x_train.x7)
sns.histplot(ax=axes[1,3], x=x_train.x8)
sns.histplot(ax=axes[2,0], x=x_train.x9)
sns.histplot(ax=axes[2,1], x=x_train.x10)
sns.histplot(ax=axes[2,2], x=x_train.x11)
sns.histplot(ax=axes[2,3], x=x_train.x12)
sns.histplot(ax=axes[3,0], x=x_train.x13)
sns.histplot(ax=axes[3,1], x=x_train.x14)
sns.histplot(ax=axes[3,2], x=x_train.x15)
sns.histplot(ax=axes[3,3], x=x_train.x16)
sns.histplot(ax=axes[4,0], x=x_train.x17)
sns.histplot(ax=axes[4,1], x=x_train.x18)
sns.histplot(ax=axes[4,2], x=x_train.x19)
sns.histplot(ax=axes[4,3], x=x_train.x20)

จากกราฟด้านบนพบว่าแทบทุก feature มีการแจกแจกปกติ ยกเว้น X2 ที่มีการเบ้เล็กน้อย และ X9 ที่มีความหนาแน่นตรงค่า 0 สูงขึ้นมา จึงเลือกใช้ค่า median ในการจัดการกับข้อมูล N/A

In [ ]:
x_train_fillna = x_train.fillna(x_train.median())
x_train_fillna.info()

เนื่องจาก scale ของแต่ละ feature ยังมีขนาดต่างกันมากอยู่ ดูได้จากค่า mean, median, min หรือ maxของแต่ละ features ที่มีค่าต่างกันมาก และเนื่องจากข้อมมูลส่วนใหญ่มีการแจกแจงแบบปกติจึงจะปรับ scale ของข้อมูลด้วยวิธี z score normalization เพื่อให้ข้อมูลในทุก feature มีขนาดเดียวกัน

In [ ]:
x_train_norm = zscore(x_train_fillna)
x_train_norm.describe().T

## 2. Exploratory Data Analysis (describe insight and visualization)

จำนวน classification (y) ของจำนวนข้อมูล train set

In [ ]:
sns.countplot( x='y' , data=df_train)

ตรวจสอบการแจกแจงของข้อมูลเปรียบเทียบการเปลี่ยนแปลงอีกครั้งระหว่าง ข้อมูลเริ่มต้น, ข้อมูลเมื่อจัดการช่องว่างด้วย median และข้อมูลเมื่อทำการ normalization

In [ ]:
fig, axes = plt.subplots(20, 3,figsize=(20, 40))
# raw data
sns.histplot(ax=axes[0,0], x=x_train.x1)
axes[0,0].set_title('raw data frame')
axes[0,0].set_ylabel('X1')
# fill na data
sns.histplot(ax=axes[0,1], x=x_train_fillna.x1)
axes[0,1].set_title('fill na with median')
axes[0,1].set_ylabel('X1')
# normalization data
sns.histplot(ax=axes[0,2], x=x_train_norm.x1)
axes[0,2].set_title('z-score normalization')
axes[0,2].set_ylabel('X1')

sns.histplot(ax=axes[1,0], x=x_train.x2)
axes[1,0].set_ylabel('X2')
sns.histplot(ax=axes[1,1], x=x_train_fillna.x2)
axes[1,1].set_ylabel('X2')
sns.histplot(ax=axes[1,2], x=x_train_norm.x2)
axes[1,2].set_ylabel('X2')

sns.histplot(ax=axes[2,0], x=x_train.x3)
axes[2,0].set_ylabel('X3')
sns.histplot(ax=axes[2,1], x=x_train_fillna.x3)
axes[2,1].set_ylabel('X3')
sns.histplot(ax=axes[2,2], x=x_train_norm.x3)
axes[2,2].set_ylabel('X3')

sns.histplot(ax=axes[3,0], x=x_train.x4)
axes[3,0].set_ylabel('X4')
sns.histplot(ax=axes[3,1], x=x_train_fillna.x4)
axes[3,1].set_ylabel('X4')
sns.histplot(ax=axes[3,2], x=x_train_norm.x4)
axes[3,2].set_ylabel('X4')

sns.histplot(ax=axes[4,0], x=x_train.x5)
axes[4,0].set_ylabel('X5')
sns.histplot(ax=axes[4,1], x=x_train_fillna.x5)
axes[4,1].set_ylabel('X5')
sns.histplot(ax=axes[4,2], x=x_train_norm.x5)
axes[4,2].set_ylabel('X5')

sns.histplot(ax=axes[5,0], x=x_train.x6)
axes[5,0].set_ylabel('X6')
sns.histplot(ax=axes[5,1], x=x_train_fillna.x6)
axes[5,1].set_ylabel('X6')
sns.histplot(ax=axes[5,2], x=x_train_norm.x6)
axes[5,2].set_ylabel('X6')

sns.histplot(ax=axes[6,0], x=x_train.x7)
axes[6,0].set_ylabel('X7')
sns.histplot(ax=axes[6,1], x=x_train_fillna.x7)
axes[6,1].set_ylabel('X7')
sns.histplot(ax=axes[6,2], x=x_train_norm.x7)
axes[6,2].set_ylabel('X7')

sns.histplot(ax=axes[7,0], x=x_train.x8)
axes[7,0].set_ylabel('X8')
sns.histplot(ax=axes[7,1], x=x_train_fillna.x8)
axes[7,1].set_ylabel('X8')
sns.histplot(ax=axes[7,2], x=x_train_norm.x8)
axes[7,2].set_ylabel('X8')

sns.histplot(ax=axes[8,0], x=x_train.x9)
axes[8,0].set_ylabel('X9')
sns.histplot(ax=axes[8,1], x=x_train_fillna.x9)
axes[8,1].set_ylabel('X9')
sns.histplot(ax=axes[8,2], x=x_train_norm.x9)
axes[8,2].set_ylabel('X9')

sns.histplot(ax=axes[9,0], x=x_train.x10)
axes[9,0].set_ylabel('X10')
sns.histplot(ax=axes[9,1], x=x_train_fillna.x10)
axes[9,1].set_ylabel('X10')
sns.histplot(ax=axes[9,2], x=x_train_norm.x10)
axes[9,2].set_ylabel('X10')

sns.histplot(ax=axes[10,0], x=x_train.x11)
axes[10,0].set_ylabel('X11')
sns.histplot(ax=axes[10,1], x=x_train_fillna.x11)
axes[10,1].set_ylabel('X11')
sns.histplot(ax=axes[10,2], x=x_train_norm.x11)
axes[10,2].set_ylabel('X11')

sns.histplot(ax=axes[11,0], x=x_train.x12)
axes[11,0].set_ylabel('X12')
sns.histplot(ax=axes[11,1], x=x_train_fillna.x12)
axes[11,1].set_ylabel('X12')
sns.histplot(ax=axes[11,2], x=x_train_norm.x12)
axes[11,2].set_ylabel('X12')

sns.histplot(ax=axes[12,0], x=x_train.x13)
axes[12,0].set_ylabel('X13')
sns.histplot(ax=axes[12,1], x=x_train_fillna.x13)
axes[12,1].set_ylabel('X13')
sns.histplot(ax=axes[12,2], x=x_train_norm.x13)
axes[12,2].set_ylabel('X13')

sns.histplot(ax=axes[13,0], x=x_train.x14)
axes[13,0].set_ylabel('X14')
sns.histplot(ax=axes[13,1], x=x_train_fillna.x14)
axes[13,1].set_ylabel('X14')
sns.histplot(ax=axes[13,2], x=x_train_norm.x14)
axes[13,2].set_ylabel('X14')

sns.histplot(ax=axes[14,0], x=x_train.x15)
axes[14,0].set_ylabel('X15')
sns.histplot(ax=axes[14,1], x=x_train_fillna.x15)
axes[14,1].set_ylabel('X15')
sns.histplot(ax=axes[14,2], x=x_train_norm.x15)
axes[14,2].set_ylabel('X15')

sns.histplot(ax=axes[15,0], x=x_train.x16)
axes[15,0].set_ylabel('X16')
sns.histplot(ax=axes[15,1], x=x_train_fillna.x16)
axes[15,1].set_ylabel('X16')
sns.histplot(ax=axes[15,2], x=x_train_norm.x16)
axes[15,2].set_ylabel('X16')

sns.histplot(ax=axes[16,0], x=x_train.x17)
axes[16,0].set_ylabel('X17')
sns.histplot(ax=axes[16,1], x=x_train_fillna.x17)
axes[16,1].set_ylabel('X17')
sns.histplot(ax=axes[16,2], x=x_train_norm.x17)
axes[16,2].set_ylabel('X17')

sns.histplot(ax=axes[17,0], x=x_train.x18)
axes[17,0].set_ylabel('X18')
sns.histplot(ax=axes[17,1], x=x_train_fillna.x18)
axes[17,1].set_ylabel('X18')
sns.histplot(ax=axes[17,2], x=x_train_norm.x18)
axes[17,2].set_ylabel('X18')

sns.histplot(ax=axes[18,0], x=x_train.x19)
axes[18,0].set_ylabel('X19')
sns.histplot(ax=axes[18,1], x=x_train_fillna.x19)
axes[18,1].set_ylabel('X19')
sns.histplot(ax=axes[18,2], x=x_train_norm.x19)
axes[18,2].set_ylabel('X19')

sns.histplot(ax=axes[19,0], x=x_train.x20)
axes[19,0].set_ylabel('X20')
sns.histplot(ax=axes[19,1], x=x_train_fillna.x20)
axes[19,1].set_ylabel('X20')
sns.histplot(ax=axes[19,2], x=x_train_norm.x20)
axes[19,2].set_ylabel('X20')

พบว่าการแจกแจงของข้อมูลทุก feature หลังจากการจัดการข้อมูลที่ผ่านมายังคงสภาพเดิม และขนาด scale ของทุก feature มีค่าที่ช่วงเดียวกันแล้ว<br>
จากนั้นจะใช้กราฟ correlation เพื่อดูความสัมพันธ์ของ feature แต่ละตัวเปรียบเทียบกันระหว่าง ข้อมูลเริ่มต้น, ข้อมูลเมื่อจัดการช่องว่างด้วย median และข้อมูลเมื่อทำการ normalization เพื่อจะรักษาคุณสมบัติของ feature ให้ตามเดิม

In [ ]:
fig, axes = plt.subplots(1, 3,figsize=(20, 5))
sns.heatmap(df_train.corr(), cmap="PiYG", ax=axes[0])
axes[0].set_title('raw data')
sns.heatmap(x_train_fillna.corr(), cmap="PiYG", ax=axes[1])
axes[1].set_title('fill na with median')
sns.heatmap(x_train_norm.corr(), cmap="PiYG", ax=axes[2])
axes[2].set_title('z-score normalization')

## 3. Training data - Crossvalidation

นำข้อมูลมา train ด้วย MLPClassifier โดยใช้ Optimiser Algorithm คือ Adaptive Moment Estimation (adam) <br>
train โดยการใช้ Crossvalidation: k-fold = 5 เพื่อดูค่าประสิทธิภาพของผลลัพธ์ที่ได้ทุกชุดของการ train และความแปรปรวนของประสิทธิภาพโมเดล

In [ ]:
clf = MLPClassifier(random_state=0, max_iter=2000, activation = 'logistic', alpha = 0.05
                   ,beta_1 = 0.9, beta_2 = 0.9999, epsilon = 1e-9, n_iter_no_change = 10)
KFold = KFold(n_splits=5, shuffle=False)
kfold = KFold.split(x_train_norm, y_train)
scores = []

for k, (train, test) in enumerate(kfold):
    clf.fit(x_train_norm.iloc[train, :], y_train.iloc[train])
    score = clf.score(x_train_norm.iloc[test, :], y_train.iloc[test])
    scores.append(score)
    print('Fold: %2d, Training/Test Split Distribution: %s, Accuracy: %.3f' % (k+1, np.bincount(y_train.iloc[train]), score))
 
print('\n\nCross-Validation accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

score ของ model มีค่าเฉลี่ยของ 5 รอบคือ 0.983 และมีความแปรปรวนคือ 0.003<br>
จากนั้นลองตรวจสอบอีกครั้งโดยใช้ model ที่ได้มาทำนาย และดูประสิทธิภาพกับข้อมูล train set ทั้งหมด 7,500 instance

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

test = confusion_matrix(y_train, clf.predict(x_train_norm))
print(test)
print(test.shape) 
tn, fp, fn, tp = confusion_matrix(y_train, clf.predict(x_train_norm)).ravel()
print("tp=",tp," fp=",fp)
print("fn=",fn," tn=", tn)

precision = metrics.precision_score(y_train, clf.predict(x_train_norm))
recall = metrics.recall_score(y_train, clf.predict(x_train_norm))
f1 = metrics.f1_score(y_train, clf.predict(x_train_norm))
f1_weighted = metrics.f1_score(y_train, clf.predict(x_train_norm), average = 'weighted')
accuracy = metrics.accuracy_score(y_train, clf.predict(x_train_norm))

print("accuracy = ", accuracy)
print("precision = ",precision)
print("recall = ", recall)
print("f1 = ", f1)
print("f1 weighted = ", f1_weighted)
print(metrics.classification_report(y_train, clf.predict(x_train_norm)))

## 4. Testing data

เตรียมข้อมูล test set เพื่อนำ model ที่ได้ไปทำนาย classification 

In [ ]:
df_test.info()

In [ ]:
df_test.describe().T

In [ ]:
df_test = zscore(df_test)
df_test.describe().T

In [ ]:
# output
predict = clf.predict(df_test)
fields = ['id','Expected']
df = pd.DataFrame()
id = list(range(1,2501))
df['id']=pd.Series(id)
df['Expected']=pd.Series(predict)
df.to_csv('submit3.csv', index=False)